In [9]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
from pgmpy.estimators import HillClimbSearch
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BicScore
from pgmpy.base import DAG
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from pgmpy.base import DAG
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
from rpy2.robjects import globalenv
from rpy2.robjects.packages import importr
from pgmpy.estimators import K2Score
from rpy2.robjects import conversion, default_converter, pandas2ri
from rpy2.robjects.conversion import localconverter
from pgmpy.models import BayesianNetwork
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
import rpy2.robjects as ro
from pgmpy.estimators import ExpectationMaximization as EM
from pgmpy.estimators import BayesianEstimator as BE
pandas2ri.activate()

In [10]:
def to_matrix(edges_dag,columns):
    #将图转换为矩阵
    edges_matrix = pd.DataFrame(np.zeros((len(columns),len(columns))),index = columns,columns=columns).astype(int)
    for edges in edges_dag:
        edges_matrix.loc[edges[0],edges[1]]+=1
    return edges_matrix
def to_dag(edges_matrix):
    edges_list = []
    for index in edges_matrix.index:
        for col in edges_matrix.columns:
            if edges_matrix.loc[index,col]!=0:
                edges_list.append((index,col))
            
    #将矩阵转换为图
    return edges_list
# Funtion to evaluate the learned model structures.
def get_f1_score(estimated_model, true_model):
    nodes = estimated_model.nodes()
    est_adj = nx.to_numpy_matrix(
        estimated_model.to_undirected(), nodelist=nodes, weight=None
    )
    true_adj = nx.to_numpy_matrix(
        true_model.to_undirected(), nodelist=nodes, weight=None
    )

    f1 = f1_score(np.ravel(true_adj), np.ravel(est_adj))
    print("F1-score for the model skeleton: ", f1)    
def print_dag(dag):
    if dag.has_node("label"):
        for edges in dag.get_ancestral_graph("label").edges:
            print(edges[0],"->",edges[1],";")
    else:
        for edges in dag.edges:
            print(edges[0],"->",edges[1],";")
def model_auc(prob,y_true):
    y_prob = []
    num = 0
    for i in range(len(prob)):
        y_prob.append(prob.iloc[i,y_true[i]])
    if y_predic[i]==y_true[i]:
        num+=1
    fpr, tpr, thresholds = roc_curve(y_true,y_predic)
    
    return auc(fpr, tpr)


In [11]:
class client():
    """
    只改动模型训练部分
    black和whitelist 的名单修改
    
    """
    def __init__(self,data,cid):
        #black 用R语言的存储方式存储·
        self.data = data
        self.cid = cid
        #初始化创建一个空的图
        self.model = DAG()
    def get_model(self):
        #获得本地客户端网络结构
        return self.model
    def fit(self):
        #记录最好的模型
        # 代改动部分        
        
        self_r_data = ro.conversion.py2rpy(self.data)
        globalenv['self_r_data'] = self_r_data
        r_script="""
        library(bnlearn)
        
        
        datacols = names(self_r_data)
        for (i in 1:ncol(self_r_data)) { 
        if(i>45){
            self_r_data[,datacols[i]] <- factor(self_r_data[,datacols[i]],levels = c(0,1))
        }else{
            self_r_data[,datacols[i]] <- factor(self_r_data[,datacols[i]])
              }
        }
        dag = hc(self_r_data,score = sc,blacklist = bl_r)
        arcs = arc.strength(dag,self_r_data)
        
        """
        edges = r(r_script)
        self.model_edges = edges
        #创建新的模型，并添加结果
        self.model = DAG()
        for i in range(len(edges["from"])):
            self.model.add_edge(edges["from"][i],edges["to"][i])
      
        
        return 1
    def set_parment(self):
        #从服务器中心get到网络结构
        #初始化model
        model.edges
    
        return 
    def update_model(self,fuse_dag):
        #代改动部分，每次也是创建一个新的贝叶斯网络实例进行白名单结构学习，并返回边模型，最好还是保持pgmpy的形式
        
        
        self.model = DAG()
        #设置白名单
        white_list = fuse_dag.edges()
        wl = []
        for edge in white_list:
            wl.append([edge[0],edge[1]])
        
        wl = pd.DataFrame(wl,columns=["from","to"])
        wl_r =ro.conversion.py2rpy(wl)
        globalenv['wl_r'] = wl_r
        self_r_data = ro.conversion.py2rpy(self.data)
        globalenv['self_r_data'] = self.data
        r_script="""
        library(bnlearn)
       
        datacols = names(self_r_data)
        for (i in 1:ncol(self_r_data)) { 
        if(i>45){
            self_r_data[,datacols[i]] <- factor(self_r_data[,datacols[i]],levels = c(0,1))
        }else{
            self_r_data[,datacols[i]] <- factor(self_r_data[,datacols[i]])
              }
        }
        dag = hc(self_r_data,score = sc,blacklist = bl_r,whitelist = wl_r)
        arcs = arc.strength(dag,self_r_data)
        
        """
        edges = r(r_script)
        self.model_edges = edges
      
        for i in range(len(edges["from"])):
            self.model.add_edge(edges["from"][i],edges["to"][i])
        

        
class sever():
    def __init__(self,fuse_num):
        self.fuse_num = fuse_num
    def get_edges_matrix(self,edges_matrix):
        self.edges_matrix = edges_matrix
    def Fd_caculate(self):
        for index in  self.edges_matrix.index:
            for col in  self.edges_matrix.columns:
                if self.edges_matrix.loc[index,col]>=self.fuse_num:
                     self.edges_matrix.loc[index,col] = 1
                else :
                     self.edges_matrix.loc[index,col] = 0
        fuse_edges = to_dag(edges_matrix)
        fuseDAG = DAG()
        fuseDAG.add_edges_from(fuse_edges)
        return  fuseDAG

In [12]:
DB_1 = pd.read_csv("D:/实验室/数据/eicu数据处理/Imp6_cart_2.0.csv").drop(columns="Unnamed: 0",axis = 1)

# DB_1 = pd.read_csv("c:data/site_16topaucoutlinePmmImpData_dis.csv").drop(columns="Unnamed: 0",axis = 1)
DB_1["label"].value_counts()
DB_1 = DB_1.astype(np.int16)

# 数据分割
hos_id = [420,142,122,435,390,227,195,243,403,141]
client_data = []
for hosid in hos_id:
    client_data.append(DB_1[DB_1["hospitalid"]==hosid ].iloc[:,:].drop(columns = ["Unnamed: 0.1","hospitalid"],axis = 1))
#     client_data.append(DB_1[DB_1["hospitalid"]==hosid ].iloc[:,:].drop(columns = ["hospitalid"],axis = 1))
    
feature = client_data[0].columns

#设置黑名单
bl = []
for i in feature:
    if i =="label":
        pass
    bl.append(["label",i])
for i in feature:
    if i =="age":
        pass
    bl.append([i,"age"])
for i in feature:
    if i =="sex":
        pass
    bl.append([i,"sex"])
for i in feature:
    if i =="race":
        pass
    bl.append([i,"race"])

bl = pd.DataFrame(bl,columns=["from","to"])
bl_r = ro.conversion.py2rpy(bl)
globalenv['bl_r'] = bl_r

In [13]:
def cross_validation(ed,DB,cid):
    
    print_edges(ed)
    #cid客户端号，寻找相应的训练数据
    auc_score = np.zeros((10,1))
    recall = np.zeros((10,1))
    acc = np.zeros((10,1))
    #分割数据集
    train_data,test_data = train_test_split(DB[cid],test_size=0.2)
   
    for i in range(len(DB)): 
        model= BayesianNetwork()     
        model.add_edges_from(ed)
       
        data = DB[i]
        f_1 = model.nodes
        f_2 = data.columns
        drop_name = []
        for j in f_2:
            if not j in f_1:
                drop_name.append(j)
        data = data.drop(columns = drop_name,axis = 1)
         #训练模型
        
        
        train_data,test_data = train_test_split(data,test_size=0.3,random_state=2)
        #得到y-true
        model.fit(train_data,estimator=BE)
       #获得测试集真实结果
        y_true = test_data.iloc[:,0]
        #丢弃label获得
        
        test = test_data.drop(columns="label",axis =1)
      
        #将预测的值从df转换为list形式
        #在predict的时候不知道什么会影响结果

        y_pred = list(model.predict(test).iloc[:,0])
       
       #模型预测
        acc[i] = accuracy_score(y_true,y_pred)
        recall[i] = recall_score(y_true, y_pred)
        auc_score[i] = roc_auc_score(y_true,y_pred)

    return acc,recall,auc_score

In [14]:
def print_edges(ed):
    for i in ed:
        print(i[0],"->",i[1],";")

In [15]:
globalenv['sc'] = "aic"
pandas2ri.activate()
# 结构训练
train_round = 10
round_num = 0
client_num = 10
clients = []
for i in range(client_num):
# 创建客户端，并且传入数据和结构学习模型
    clients.append(client(client_data[i],i))
# 初始化服务器
sever_1 = sever(6)
edges_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
while(round_num<train_round):
    #利用二维矩阵传递边结构信息
    auc_list = []
    recall_list = []
    acc_list = []
    #开始训练
    #重置矩阵
    edges_matrix.iloc[:,:] = 0
    acc = 0
    recall = 0
    acu_score = 0
#     edges_strength = pd.DataFrame(np.zeros((len(feature)*len(feature),3)),columns=["from","to","strength"]).astype(np.int16)
#     edges_strength.iloc[:,:] = 0
    #第一轮，先本地客户端建立初始模型
    if round_num == 0:
        edges_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
        print("初始化训练")
        for i in range(client_num):
            clients[i].fit()
#             model_edges = clients[i].model_edges
#             for j in range(len(model_edges)):
#                 from_node = model_edges.iloc[j]["from"]
#                 to_node = model_edges.iloc[j]["to"]
#                 strength  = model_edges.iloc[j]["strength"]
#                 edges_strength.loc[(edges_strength["from"]==from_node)&(edges_strength["to"]==to_node),"strength"]+=strength
                
#     print(edges_strength.sort_values(by="strength"))
            #计算初始性能
            print("各个客户端的初始性能")
            acc,recall,auc_score = cross_validation(clients[i].model.edges,client_data,clients[i].cid)
            auc_list.append(acu_score)
            recall_list.append(recall)
            acc_list.append(acc)
        print("初始auc",auc_list)
        print("初始recall",recall_list)
        print("初始acc",acc_list)
        auc_list = []
        recall_list = []
        acc_list = [] 
    #开始聚集客户端边的信息
    for i in range(client_num):
        #将客户端最好的模型边信息转换为矩阵形式
        edges_matrix+=to_matrix(clients[i].get_model().edges(),feature)
        
    #服务器开始汇集信息，并且将矩阵转换为Dag
#     print("未过滤的融合矩阵")
#     for index in edges_matrix.index:
#         for col in edges_matrix.columns:
#             if edges_matrix.loc[index,col]!=0:
#                 print(index,"->",col,"[label=",edges_matrix.loc[index,col],"];")
    sever_1.get_edges_matrix(edges_matrix)
    fuseDAG = sever_1.Fd_caculate()
    print("第",round_num+1,"次迭代")
    print_dag(fuseDAG)
    #更新客户端模型
    print("更 新客户端")
    for i in range(client_num):
        clients[i]. update_model(fuseDAG)
    round_num+=1
    #测试更新后的性能
#     for i in range(client_num):
#         print("各个客户端的初始性能")
#         acc,recall,auc_score = cross_validation(clients[i].model.edges,client_data,clients[i].cid)
#         auc_list.append(acu_score)
#         recall_list.append(recall)
#         acc_list.append(acc)
#     print("更新auc",auc_list)
#     print("更新recall",recall_list)
#     print("更新acc",acc_list)
    auc_list = []
    recall_list = []
    acc_list = [] 
for i in range(client_num):
    #将客户端最好的模型边信息转换为矩阵形式
    edges_matrix+=to_matrix(clients[i].get_model().edges(),feature)



初始化训练
各个客户端的初始性能
LAB43 -> LAB44 ;
LAB43 -> COM4 ;
LAB43 -> LAB11 ;
LAB44 -> LAB12 ;
LAB44 -> LAB56 ;
LAB44 -> LAB10 ;
LAB44 -> LAB46 ;
LAB53 -> LAB54 ;
LAB53 -> LAB55 ;
LAB46 -> LAB48 ;
LAB46 -> LAB47 ;
LAB46 -> LAB56 ;
LAB46 -> LAB59 ;
LAB46 -> LAB18 ;
LAB46 -> LAB35 ;
LAB46 -> LAB42 ;
LAB48 -> LAB57 ;
LAB48 -> LAB27 ;
LAB6 -> LAB43 ;
LAB6 -> LAB46 ;
LAB6 -> PRO3 ;
LAB6 -> LAB44 ;
LAB39 -> LAB42 ;
LAB39 -> COM7 ;
LAB39 -> LAB27 ;
LAB42 -> LAB11 ;
LAB42 -> LAB14 ;
LAB42 -> PRO1 ;
LAB23 -> LAB36 ;
LAB36 -> LAB50 ;
LAB2 -> LAB16 ;
LAB2 -> LAB39 ;
LAB2 -> LAB24 ;
LAB2 -> LAB42 ;
LAB2 -> LAB7 ;
LAB2 -> PRO3 ;
LAB2 -> PRO7 ;
LAB16 -> LAB39 ;
LAB16 -> LAB50 ;
LAB16 -> LAB8 ;
COM8 -> label ;
COM8 -> COM10 ;
COM8 -> COM2 ;
COM8 -> COM11 ;
COM8 -> PRO8 ;
COM8 -> COM4 ;
COM8 -> COM6 ;
COM8 -> PRO6 ;
COM8 -> PRO2 ;
COM8 -> COM9 ;
COM8 -> LAB55 ;
COM8 -> LAB3 ;
LAB15 -> LAB34 ;
LAB15 -> LAB20 ;
LAB15 -> LAB25 ;
LAB12 -> LAB59 ;
LAB12 -> LAB15 ;
LAB12 -> LAB18 ;
LAB12 -> LAB10 ;
LAB12 -> LAB35 ;
LA

  0%|          | 0/1284 [00:00<?, ?it/s]

  0%|          | 0/489 [00:00<?, ?it/s]

  0%|          | 0/744 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/338 [00:00<?, ?it/s]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
auc_list = []
recall_list =[]
acc_list = []
for i in range(10):
    acc,recall,auc = cross_validation(clients[i].model.edges,client_data,clients[i].cid)
#     acc,recall,auc = cross_validation(fuseDAG.edges,client_data,clients[i].cid)
    auc_list.append(auc)
    recall_list.append(recall)
    acc_list.append(acc)

In [61]:
for i in acc_list:
    for j in i:
        print('%.4f' % j[0])

0.8341
0.8057
0.8065
0.7417
0.8116
0.7554
0.6124
0.7111
0.7035
0.8084
0.8341
0.8057
0.8065
0.7417
0.8116
0.7554
0.6124
0.7111
0.7035
0.8084
0.8341
0.8057
0.8065
0.7417
0.8116
0.7554
0.6124
0.7111
0.7035
0.8084
0.8341
0.8057
0.8065
0.7417
0.8116
0.7554
0.6124
0.7111
0.7035
0.8084
0.8341
0.8057
0.8065
0.7417
0.8116
0.7554
0.6124
0.7111
0.7035
0.8084
0.8341
0.8057
0.8065
0.7417
0.8116
0.7554
0.6124
0.7111
0.7035
0.8084
0.8341
0.8057
0.8065
0.7417
0.8116
0.7554
0.6124
0.7111
0.7035
0.8084
0.8341
0.8057
0.8065
0.7417
0.8116
0.7554
0.6124
0.7111
0.7035
0.8084
0.8341
0.8057
0.8065
0.7417
0.8116
0.7554
0.6124
0.7111
0.7035
0.8084
0.8341
0.8057
0.8065
0.7417
0.8116
0.7554
0.6124
0.7111
0.7035
0.8084


In [64]:
for i in range(10):
    print("第",i)
    print_dag(clients[i].model)

第 0
age -> heartrate ;
age -> LAB19 ;
age -> COM1 ;
heartrate -> LAB36 ;
LAB19 -> COM5 ;
LAB19 -> LAB3 ;
LAB19 -> PRO8 ;
LAB19 -> LAB53 ;
COM1 -> label ;
COM1 -> COM4 ;
COM1 -> COM5 ;
COM1 -> COM7 ;
COM1 -> COM8 ;
COM1 -> COM10 ;
COM1 -> LAB53 ;
COM1 -> LAB6 ;
COM1 -> LAB56 ;
COM5 -> COM7 ;
COM5 -> COM10 ;
COM5 -> LAB55 ;
COM5 -> COM8 ;
COM5 -> COM11 ;
COM4 -> COM10 ;
COM4 -> PRO2 ;
COM4 -> COM8 ;
COM7 -> COM3 ;
COM7 -> COM11 ;
COM8 -> label ;
COM8 -> COM7 ;
COM8 -> COM11 ;
COM8 -> PRO8 ;
COM8 -> PRO7 ;
COM8 -> COM3 ;
COM8 -> LAB55 ;
COM8 -> PRO2 ;
COM8 -> LAB24 ;
COM8 -> LAB14 ;
COM10 -> label ;
COM10 -> COM3 ;
COM10 -> COM11 ;
COM10 -> COM8 ;
COM10 -> COM7 ;
sex -> LAB6 ;
sex -> LAB43 ;
sex -> LAB44 ;
sex -> COM3 ;
sex -> COM4 ;
sex -> LAB12 ;
sex -> COM1 ;
LAB6 -> LAB43 ;
LAB6 -> LAB44 ;
LAB6 -> LAB46 ;
LAB6 -> PRO3 ;
LAB43 -> LAB44 ;
LAB44 -> LAB10 ;
LAB44 -> LAB12 ;
LAB44 -> LAB46 ;
LAB44 -> LAB56 ;
LAB44 -> COM4 ;
COM3 -> label ;
LAB46 -> LAB56 ;
LAB46 -> LAB59 ;
LAB46 -> LAB57 ;

LAB3 -> COM11 ;
COM11 -> label ;
LAB4 -> LAB18 ;
LAB18 -> LAB50 ;
LAB50 -> COM8 ;
LAB47 -> PRO3 ;
LAB53 -> LAB54 ;
LAB53 -> LAB57 ;
LAB59 -> LAB10 ;
LAB59 -> PRO3 ;
LAB59 -> LAB14 ;
LAB59 -> LAB15 ;
LAB54 -> LAB23 ;
LAB14 -> LAB4 ;
LAB14 -> LAB15 ;
LAB15 -> LAB4 ;
LAB23 -> LAB36 ;
LAB23 -> LAB57 ;
LAB36 -> LAB35 ;
LAB36 -> LAB18 ;
LAB57 -> LAB18 ;
LAB57 -> LAB35 ;
LAB57 -> COM1 ;
LAB11 -> LAB3 ;
PRO3 -> label ;
PRO3 -> COM8 ;
PRO3 -> COM3 ;
PRO3 -> LAB50 ;
LAB35 -> LAB59 ;
第 7
LAB15 -> LAB4 ;
LAB19 -> LAB3 ;
LAB19 -> label ;
PRO1 -> COM8 ;
LAB53 -> LAB54 ;
COM11 -> label ;
LAB4 -> LAB3 ;
LAB4 -> COM10 ;
LAB10 -> LAB14 ;
LAB10 -> LAB53 ;
LAB14 -> LAB4 ;
LAB14 -> LAB15 ;
age -> heartrate ;
age -> LAB19 ;
age -> COM1 ;
LAB6 -> LAB43 ;
LAB6 -> LAB44 ;
LAB6 -> LAB46 ;
LAB6 -> LAB59 ;
LAB6 -> LAB12 ;
sex -> LAB6 ;
sex -> LAB43 ;
sex -> LAB44 ;
sex -> LAB57 ;
sex -> LAB23 ;
COM10 -> label ;
COM10 -> COM11 ;
LAB59 -> LAB10 ;
LAB59 -> LAB35 ;
LAB59 -> PRO1 ;
COM8 -> label ;
COM8 -> COM11 ;
COM8

In [17]:
for i,j in zip(hos_id,client_data):
    path = "C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data"+str(i)+".csv"
    print(path)
    j.to_csv(path)

C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data420.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data142.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data122.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data435.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data390.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data227.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data195.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data243.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data403.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data141.csv
